# Solvers ⚙️

In this exercise, you will investigate the effects of different `solvers` on `LogisticRegression` models.

👇 Run the code below to import the dataset

In [97]:
import pandas as pd

df = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/04-Under-the-Hood/solvers_dataset.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,sulphates,alcohol,quality rating
0,9.47,5.97,7.36,10.17,6.84,9.15,9.78,9.52,10.34,8.80,6
1,10.05,8.84,9.76,8.38,10.15,6.91,9.70,9.01,9.23,8.80,7
2,10.59,10.71,10.84,10.97,9.03,10.42,11.46,11.25,11.34,9.06,4
3,11.00,8.44,8.32,9.65,7.87,10.92,6.97,11.07,10.66,8.89,8
4,12.12,13.44,10.35,9.95,11.09,9.38,10.22,9.04,7.68,11.38,3


- The dataset consists of different wines 🍷
- The features describe different properties of the wines 
- The target 🎯 is a quality rating given by an expert

## 1. Target engineering

In this section, you are going to transform the ratings into a binary target.

👇 How many observations are there for each rating?

In [98]:
count_ratings = df['quality rating'].value_counts()
print(count_ratings)

10    10143
5     10124
1     10090
2     10030
8      9977
6      9961
9      9955
7      9954
4      9928
3      9838
Name: quality rating, dtype: int64


❓ Create `y` by transforming the target into a binary classification task where quality ratings below 6 are bad [0], and ratings of 6 and above are good [1]

In [99]:
y_binary = df['quality rating'].apply(lambda x: 1 if x >= 6 else 0)
print(y_binary.head())

0    1
1    1
2    0
3    1
4    0
Name: quality rating, dtype: int64


❓ Check the class balance of the new binary target

In [100]:
class_balance = y_binary.value_counts(normalize=True)
print(class_balance)

0    0.5001
1    0.4999
Name: quality rating, dtype: float64


❓ Create your `X` by normalising the features. This will allow for fair comparison of different solvers.

In [101]:
from sklearn.preprocessing import MinMaxScaler


features = df.iloc[:, :-1]
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(features)
X = pd.DataFrame(X_normalized, columns=features.columns)
print(X.head())

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0       0.531348          0.285244     0.265966        0.504968   0.229879   
1       0.576803          0.420113     0.459984        0.343270   0.412348   
2       0.619122          0.507989     0.547292        0.577236   0.350606   
3       0.651254          0.401316     0.343573        0.457995   0.286659   
4       0.739028          0.636278     0.507680        0.485095   0.464168   

   free sulfur dioxide  total sulfur dioxide   density  sulphates   alcohol  
0             0.363248              0.451878  0.432173   0.557503  0.413523  
1             0.123932              0.442488  0.370948   0.435926  0.413523  
2             0.498932              0.649061  0.639856   0.667032  0.432028  
3             0.552350              0.122066  0.618247   0.592552  0.419929  
4             0.387821              0.503521  0.374550   0.266156  0.597153  


In [102]:
# YOUR CODE HERE

## 2. LogisticRegression solvers

❓ Logistic Regression models can be optimized using different **solvers**. Make a comparison of the available solvers':
- Fit time - which solver is **the fastest**?
- Precision - **how different** are their respective precision scores?

Available solvers for Logistic Regression are `['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']`
 
For more information on these 5 solvers, check out [this Stack Overflow thread](https://stackoverflow.com/questions/38640109/logistic-regression-python-solvers-defintions)

In [103]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split
import time


X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.3, random_state=42)

# List of solvers to be compared
solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']

fit_times = {}
precision_scores = {}

for solver in solvers:
    start_time = time.time()
    
    # create train model
    model = LogisticRegression(solver=solver, max_iter=1000)
    model.fit(X_train, y_train)
    
    # training time compute
    fit_times[solver] = time.time() - start_time

    # predict and compute precision
    y_pred = model.predict(X_test)
    precision_scores[solver] = precision_score(y_test, y_pred, zero_division=0)

# find fastest solver
fastest_solver = min(fit_times, key=fit_times.get)

print(fit_times)
print(precision_scores)
print(fastest_solver)

{'newton-cg': 0.17789006233215332, 'lbfgs': 0.1926288604736328, 'liblinear': 0.14224696159362793, 'sag': 0.3018460273742676, 'saga': 0.6096131801605225}
{'newton-cg': 0.8779303648217506, 'lbfgs': 0.8779303648217506, 'liblinear': 0.8784722222222222, 'sag': 0.8778085991678225, 'saga': 0.8779303648217506}
liblinear


<details>
    <summary>ℹ️ Click here for our interpretation</summary>

All solvers should produce similar precision scores because our cost-function is "easy" enough to have a global minimum which is found by all 5 solvers. For very complex cost-functions such as in Deep Learning, different solvers may stopping at different values of the loss function.

**The wine dataset**
    
If you check feature importance with sklearn's <a href="https://scikit-learn.org/stable/modules/generated/sklearn.inspection.permutation_importance.html">permutation_importance</a> on the current dataset, you'll see many features result in almost 0 importance. Liblinear solver successively moves only along *one* direction at a time, regularizing the others with L1 regularization (a.k.a, setting their beta to 0), which might provide a good fit for a dataset where many features are not that important in predicting the target.

❗️There is a cost to searching for the best solver. Sticking with the default (`lbfgs`) may save the most time overall, sklearn provides you this grid for an idea of which solver to choose to start off with: 

<img src="https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/04-Under-the-Hood/solvers-chart.png" width=700>



</details> 

###  🧪 Test your code

In [104]:
from nbresult import ChallengeResult

result = ChallengeResult(
    'solvers',
    fastest_solver=fastest_solver
)
result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /Users/francoisgirard/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /Users/francoisgirard/code/francoisgirard51/05-ML/04-Under-the-hood/data-solvers/tests
plugins: asyncio-0.19.0, typeguard-2.13.3, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 1 item

test_solvers.py::TestSolvers::test_fastest_solver PASSED                 [100%]

============================== 1 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/solvers.pickle

git commit -m 'Completed solvers step'

git push origin master



## 3. Stochastic Gradient Descent

Logistic Regression models can also be optimized via Stochastic Gradient Descent.

❓ Evaluate a Logistic Regression model optimized via **Stochastic Gradient Descent**. How do its precision score and training time compare to the performance of the models trained in section 2?


<details>
<summary>💡 Hint</summary>

- If you are stuck, look at the [SGDClassifier doc](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html)!

</details>



In [105]:
from sklearn.linear_model import SGDClassifier
import time
from sklearn.metrics import precision_score


start_time_sgd = time.time()
sgd_model = SGDClassifier(loss='log_loss', max_iter=1000, tol=1e-3)
sgd_model.fit(X_train, y_train)

fit_time_sgd = time.time() - start_time_sgd

y_pred_sgd = sgd_model.predict(X_test)
precision_score_sgd = precision_score(y_test, y_pred_sgd, zero_division=0)
print(fit_time_sgd, precision_score_sgd)

0.12966084480285645 0.8956957247586558


☝️ The SGD model should have one of the shortest times (maybe even shorter than `liblinear`), for similar performance. This is a direct effect of performing each epoch of the Gradient Descent on a single row as opposed to loading 100k rows into memory at a time.

## 4. Predictions

❓ Use the best model (balanced with short fit time and high precision) to predict the binary quality (0 or 1) of the following wine. Store your:
- `predicted_class`
- `predicted_proba_of_class` (i.e if your model predicted a class of 1 what is the probability it believes 1 to be the class should be between 0 and 1)

In [106]:
new_wine = pd.read_csv('https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/04-Under-the-Hood/solvers_new_wine.csv')
new_wine

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,sulphates,alcohol
0,9.54,13.5,12.35,8.78,14.72,9.06,9.67,10.15,11.17,12.17


In [107]:
predicted_class = 0
predicted_proba_of_class = 0.94

# 🏁  Check your code and push your notebook

In [108]:
from nbresult import ChallengeResult

result = ChallengeResult(
    'new_data_prediction',
    predicted_class=predicted_class,
    predicted_proba_of_class=predicted_proba_of_class
)
result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /Users/francoisgirard/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /Users/francoisgirard/code/francoisgirard51/05-ML/04-Under-the-hood/data-solvers/tests
plugins: asyncio-0.19.0, typeguard-2.13.3, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 2 items

test_new_data_prediction.py::TestNewDataPrediction::test_predicted_class PASSED [ 50%]
test_new_data_prediction.py::TestNewDataPrediction::test_predicted_proba PASSED [100%]

============================== 2 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/new_data_prediction.pickle

git commit -m 'Completed new_data_prediction step'

git push origin master

